In [1]:
import happybase
import pandas as pd
import findspark
findspark.init()

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, from_unixtime
from functools import reduce

In [3]:
VM_adress = 'localhost'
connection = happybase.Connection(VM_adress, timeout=999999)
dynamic_table = connection.table('daily_data')
static_table = connection.table('spotify_songs')

In [4]:
spark = SparkSession.builder.appName("SpotifyAnalysis").getOrCreate()

23/12/29 23:29:29 WARN util.Utils: Your hostname, node1 resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s3)
23/12/29 23:29:29 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/12/29 23:29:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
def create_spark_df(tab, spark):
    data_list = []
    
    for key, data in tab.scan(None):
        key_decoded = key.decode('utf-8')
        row_dict = {'id': key_decoded}
    
        for column, value in data.items():
            column_decoded = column.decode('utf-8')
            value_decoded = value.decode('utf-8')
            row_dict[column_decoded] = value_decoded
    
        data_list.append(row_dict)
        
    spark_df = spark.createDataFrame(data_list)
    return spark_df

In [7]:
daily_data = create_spark_df(dynamic_table, spark)
spotify_songs = create_spark_df(static_table, spark)

In [11]:
daily_data = reduce(lambda df, col_name: df.withColumnRenamed(col_name, col_name.replace("data:", "")), daily_data.columns, daily_data)
daily_data.show()

+-------------+-------------------+----------+----+--------------------+--------------------+
|       artist|         date_added|popularity|rank|          track_name|                  id|
+-------------+-------------------+----------+----+--------------------+--------------------+
|   Elley Duhé|2023-12-29T09:09:03|        78|  39|MONEY ON THE DASH...|086THPnabbu1zfDjR...|
|  Blacha 2115|2023-12-29T09:09:03|        65|   2|    Kevin sam w domu|0DhKkQjcrOXLBr9eC...|
|        CYRIL|2023-12-29T09:09:03|        68|  32|        Stumblin' In|0h3Xy4V4apMraB5Nu...|
|      PRO8L3M|2023-12-29T09:09:03|        66|  47|   Byłem człowiekiem|0nQljvBR8fuT0nmLK...|
|     Smolasty|2023-12-29T09:09:03|        69|  15|  Nim Zajdzie Słońce|1NtvjagHJRLQogzWV...|
|        Fukaj|2023-12-29T09:09:03|        65|  24|          NIE MA CIĘ|1cnkQScXEGBZKqqy6...|
|       Pusher|2023-12-29T09:09:03|        64|  38|           Leje Wina|1feg6LWYEdvsjOLzA...|
|         Kizo|2023-12-29T09:09:03|        60|  31|         

In [13]:
spotify_songs = reduce(lambda df, col_name: df.withColumnRenamed(col_name, col_name.replace("data:", "")), spotify_songs.columns, spotify_songs)
spotify_songs.show()

23/12/29 23:34:54 WARN scheduler.TaskSetManager: Stage 3 contains a task of very large size (4021 KiB). The maximum recommended task size is 1000 KiB.


+------------+------------+-----------+------+----------------+---+--------+--------+----+--------------+----------------------+--------------------------------------------------------------------------------------------------+-------------------------+-----------+-------+----------------------+--------------------------------------------+------------------------+--------------------+--------------------------------------------+----------------+-------+----------------------+
|acousticness|danceability|duration_ms|energy|instrumentalness|key|liveness|loudness|mode|playlist_genre|playlist_id           |playlist_name                                                                                     |playlist_subgenre        |speechiness|tempo  |track_album_id        |track_album_name                            |track_album_release_date|track_artist        |track_name                                  |track_popularity|valence|id                    |
+------------+------------+-----------